In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, DoubleType,StructField, StructType


In [5]:
conf = SparkConf().setMaster("local[3]").setAppName("partitioning_shuffle").set("spark.jars.packages", "com.crealytics:spark-excel_2.11:0.12.2")
sc = SparkContext(conf=conf).getOrCreate()
spark = SparkSession(sc)
spark

In [ ]:
df = spark.read.format("com.crealytics.spark.excel") \
.option("useHeader", "true") \
.option("inferSchema", "true") \
.load("C:\\Users\\EdwinVivekN\\Desktop\\sample files\\test.xlsx")

In [7]:
df = spark.read.option("header", "true") \
.option("inferSchema", "true")\
.csv("C:\\Users\\EdwinVivekN\Desktop\\sample files\\NEW_All_AppleTV_PrimeVideo_RevenueTrackin.csv")

In [10]:
df.printSchema()
df.count()
df.describe('Unit Price').show()


#change data type
changedTypedf = df.withColumn("Row ID", df["Row ID"].cast(DoubleType()))
changedTypedf

root
 |-- Row ID: string (nullable = true)
 |-- Title/Season ID: string (nullable = true)
 |-- ASIN: string (nullable = true)
 |-- Title Name: string (nullable = true)
 |-- Series Name: string (nullable = true)
 |-- Season Name: string (nullable = true)
 |-- Consumption Type: string (nullable = true)
 |-- Quality: string (nullable = true)
 |-- Unit Price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Duration Streamed: string (nullable = true)
 |-- Paid Ad Impressions: string (nullable = true)
 |-- Ad Revenue: string (nullable = true)
 |-- Royalty Rate: double (nullable = true)
 |-- Royalty Amount: string (nullable = true)
 |-- Royalty Currency: string (nullable = true)
 |-- Exchange rate: string (nullable = true)
 |-- $USD Royalty Amount: string (nullable = true)
 |--  Estimated $USD GROSS revenues : string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Territory: string (nullable = true)
 |-- Period Start: string (nullable = true)
 |-- Period En

7962

+-------+------------------+
|summary|        Unit Price|
+-------+------------------+
|  count|              4515|
|   mean|5.5489700996681455|
| stddev| 3.577174063761296|
|    min|               0.0|
|    max|             15.99|
+-------+------------------+



3

DataFrame[Row ID: double, Title/Season ID: string, ASIN: string, Title Name: string, Series Name: string, Season Name: string, Consumption Type: string, Quality: string, Unit Price: double, Quantity: int, Duration Streamed: string, Paid Ad Impressions: string, Ad Revenue: string, Royalty Rate: double, Royalty Amount: string, Royalty Currency: string, Exchange rate: string, $USD Royalty Amount: string,  Estimated $USD GROSS revenues : string, Region: string, Territory: string, Period Start: string, Period End: string, Type: string, CER Percentile: string, _c25: string]

In [13]:
#Distinct count of 1 field

df.select('Quality').distinct().count()


#Distinct count of all fields
rows = []
for c in df.columns:
    rows.append([c, df.select(c).distinct().count()])
df2 = spark.createDataFrame(rows, ["Colunm Name", "Distinct Count"])
df2.show()

+--------------------+--------------+
|         Colunm Name|Distinct Count|
+--------------------+--------------+
|              Row ID|          7962|
|     Title/Season ID|           266|
|                ASIN|          1104|
|          Title Name|           255|
|         Series Name|             5|
|         Season Name|             5|
|    Consumption Type|             3|
|             Quality|             3|
|          Unit Price|            88|
|            Quantity|           326|
|   Duration Streamed|          2793|
| Paid Ad Impressions|             1|
|          Ad Revenue|             1|
|        Royalty Rate|             4|
|      Royalty Amount|          1719|
|    Royalty Currency|             4|
|       Exchange rate|            25|
| $USD Royalty Amount|          2574|
| Estimated $USD G...|          2369|
|              Region|             5|
+--------------------+--------------+
only showing top 20 rows



In [12]:
df.groupBy('Title/Season ID').agg({"Row ID":'count'}).show()

+---------------+-------------+
|Title/Season ID|count(Row ID)|
+---------------+-------------+
|    8Z1Q3WNG809|          152|
|    T88GM85KJ6M|           30|
|    8QCXQQ7S13K|           28|
|    RB0662ZDVHD|           14|
|    GQ08HR5MC5H|            3|
|    ESSAG5226FC|            4|
|    63KGWNZ6PCV|            3|
|    T6DMTK8PGZJ|            7|
|    T7HWTHBJD56|           28|
|    CGCKK4PWRPE|           15|
|    P36C37R961F|            4|
|    PACRYZSQZBB|           37|
|    E8Q18HZ1HA5|           18|
|    M0AAYY64AE2|            7|
|    CKAXDN66Q8Q|            4|
|    M80ZDYV1W12|           61|
|    TB73XFT6CS1|           39|
|    0JRRSR1H8AA|            1|
|    44829G8FBMV|           16|
|    P0WN74JVXXB|           14|
+---------------+-------------+
only showing top 20 rows



In [10]:
#check parameter value we can modify at runtime or not.
spark.conf.isModifiable("spark.executor.memory")
spark.conf.isModifiable("spark.sql.shuffle.partitions")

#set conf
spark.conf.get("spark.sql.shuffle.partitions")
spark.conf.set("spark.sql.shuffle.partitions", 3)
spark.conf.get("spark.sql.shuffle.partitions")

False

True

'200'

'3'

In [8]:
#method to get data count under each partition
from pyspark.sql.functions import spark_partition_id

def getPartitionCount(table):
    ttl_num_partitions = table.withColumn("partitionid",spark_partition_id()).select("partitionid").groupBy("partitionid").agg({'partitionid': "count"})
    ttl_num_partitions.show()

In [16]:
#current partition count
df.rdd.getNumPartitions()

#get repartitioned count
data = df.repartition(2)
data.rdd.getNumPartitions()

getPartitionCount(data)

1

2

+-----------+------------------+
|partitionid|count(partitionid)|
+-----------+------------------+
|          0|              3981|
|          1|              3981|
+-----------+------------------+



In [ ]:
#partitionBy()

In [ ]:
#RDDs with same paritition count, key and type will produce same partitions
dept = [("Finance",10),("Marketing",20),("Sales",30),("Sales2",30), ("Sales3",30), ("Sales4",30), ("IT",40), ("IT2",40), ("IT3",40)]
a_rdd = sc.parallelize(dept)
#a.getNumPartitions()
a_df = a_rdd.toDF()
#a_df.show()
a_df.withColumn("partitionid",spark_partition_id()).show()

ra_df = a_df.repartition("_2")
ra_df.withColumn("partitionid",spark_partition_id()).show()



a2_rdd = sc.parallelize(dept)
a_df2 = a2_rdd.toDF()

radf2 = a_df2.repartition("_2")
radf2.withColumn("partitionid",spark_partition_id()).show()


#compare both the table partitions
getPartitionCount(ra_df)
getPartitionCount(radf2)

In [19]:

ra_df.withColumn("partitionid",spark_partition_id()).show()



+---------+---+-----------+
|       _1| _2|partitionid|
+---------+---+-----------+
|    Sales| 30|          0|
|   Sales2| 30|          0|
|   Sales3| 30|          0|
|   Sales4| 30|          0|
|Marketing| 20|          1|
|       IT| 40|          1|
|      IT2| 40|          1|
|      IT3| 40|          1|
|  Finance| 10|          2|
+---------+---+-----------+



In [21]:

ra_df.show()

+---------+---+
|       _1| _2|
+---------+---+
|    Sales| 30|
|   Sales2| 30|
|   Sales3| 30|
|   Sales4| 30|
|Marketing| 20|
|       IT| 40|
|      IT2| 40|
|      IT3| 40|
|  Finance| 10|
+---------+---+



In [23]:
ra_df.count()

9